In [ ]:
import json
import numpy as np
from collections import defaultdict, Counter

file_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_sampled_4to8.jsonl"

# 初始化统计结构
stats = defaultdict(lambda: {"visual_steps": [], "text_steps": []})

# 读取数据
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        ds = item.get("data_source", "unknown")
        steps = item.get("total_steps")

        # 有些 total_steps 是字符串，统一转换为整数
        if isinstance(steps, str):
            try:
                steps = int(steps)
            except ValueError:
                continue

        if "visual_demo" in item:
            stats[ds]["visual_steps"].append(steps)
        if "text_demo" in item:
            stats[ds]["text_steps"].append(steps)

# 打印统计信息
for ds, data in stats.items():
    print(f"\n📁 Data Source: {ds}")
    for key, arr in data.items():
        if not arr:
            continue
        arr_np = np.array(arr)
        print(f"  ▶ {key}:")
        print(f"     Count: {len(arr)}")
        print(f"     Min: {arr_np.min()}")
        print(f"     Max: {arr_np.max()}")
        print(f"     Mean: {arr_np.mean():.2f}")
        print(f"     Median: {np.median(arr_np):.2f}")
        print(f"     Distribution (step count frequencies): {dict(Counter(arr))}")



📁 Data Source: h5_agilex_3rgb
  ▶ visual_steps:
     Count: 7622
     Min: 4
     Max: 11
     Mean: 7.63
     Median: 8.00
     Distribution (step count frequencies): {9: 1412, 7: 1895, 10: 540, 8: 2242, 5: 564, 6: 873, 11: 61, 4: 35}
  ▶ text_steps:
     Count: 2226
     Min: 3
     Max: 10
     Mean: 6.72
     Median: 7.00
     Distribution (step count frequencies): {7: 624, 6: 536, 5: 262, 8: 423, 9: 172, 4: 147, 10: 50, 3: 12}

📁 Data Source: h5_ur_1rgb
  ▶ visual_steps:
     Count: 9595
     Min: 1
     Max: 9
     Mean: 3.21
     Median: 3.00
     Distribution (step count frequencies): {3: 5059, 1: 94, 4: 1306, 5: 797, 2: 2000, 6: 223, 7: 40, 9: 52, 8: 24}

📁 Data Source: h5_franka_3rgb
  ▶ visual_steps:
     Count: 7596
     Min: 2
     Max: 9
     Mean: 4.66
     Median: 5.00
     Distribution (step count frequencies): {4: 3057, 5: 3865, 6: 186, 3: 192, 7: 141, 9: 26, 8: 114, 2: 15}
  ▶ text_steps:
     Count: 2541
     Min: 3
     Max: 9
     Mean: 4.66
     Median: 5.00
   

In [16]:
import json
from collections import Counter, defaultdict

def analyze_step_distribution_vt(jsonl_path, by_source=False):
    """
    分别统计 visual_demo 和 text_demo 的 total_steps 分布
    参数:
        jsonl_path (str): JSONL 文件路径
        by_source (bool): 是否按 data_source 分组统计
    """

    stats = {
        "visual": Counter(),
        "text": Counter(),
    }
    counts = {"visual": 0, "text": 0}
    stats_by_source = defaultdict(lambda: {"visual": Counter(), "text": Counter()})

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                item = json.loads(line)
            except json.JSONDecodeError:
                continue

            steps = item.get("total_steps")
            if isinstance(steps, str):
                try:
                    steps = int(steps)
                except ValueError:
                    continue

            ds = item.get("data_source", "unknown")

            # 分别计数
            if "visual_demo" in item:
                stats["visual"][steps] += 1
                counts["visual"] += 1
                if by_source:
                    stats_by_source[ds]["visual"][steps] += 1
            if "text_demo" in item:
                stats["text"][steps] += 1
                counts["text"] += 1
                if by_source:
                    stats_by_source[ds]["text"][steps] += 1

    # 输出汇总
    print(f"📊 文件: {jsonl_path}\n")

    for mode in ["visual", "text"]:
        total = counts[mode]
        print(f"=== 🧮 {mode.upper()} DEMO 分布 (共 {total} 条) ===")
        for step, count in sorted(stats[mode].items()):
            pct = count / total * 100 if total > 0 else 0
            print(f"steps={step:<4} → {count:>6} 样本 ({pct:5.2f}%)")
        print()

    if by_source:
        print("=== 📁 按 data_source 细分 ===")
        for ds, data in stats_by_source.items():
            print(f"\n➡️ {ds}")
            for mode in ["visual", "text"]:
                total = sum(data[mode].values())
                if total == 0:
                    continue
                print(f"  ▸ {mode.upper()} ({total} 条):")
                for step, count in sorted(data[mode].items()):
                    pct = count / total * 100
                    print(f"     steps={step:<4} → {count:>5} 样本 ({pct:5.2f}%)")


In [17]:
analyze_step_distribution_vt("/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl", by_source=True)

📊 文件: /projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl

=== 🧮 VISUAL DEMO 分布 (共 98733 条) ===
steps=1    →    165 样本 ( 0.17%)
steps=2    →   3516 样本 ( 3.56%)
steps=3    →  12492 样本 (12.65%)
steps=4    →  14064 样本 (14.24%)
steps=5    →  17205 样本 (17.43%)
steps=6    →   8082 样本 ( 8.19%)
steps=7    →  10836 样本 (10.98%)
steps=8    →  10584 样本 (10.72%)
steps=9    →   7884 样本 ( 7.99%)
steps=10   →   6870 样本 ( 6.96%)
steps=11   →   3465 样本 ( 3.51%)
steps=12   →   1296 样本 ( 1.31%)
steps=13   →    741 样本 ( 0.75%)
steps=14   →    420 样本 ( 0.43%)
steps=15   →    675 样本 ( 0.68%)
steps=16   →    336 样本 ( 0.34%)
steps=17   →    102 样本 ( 0.10%)

=== 🧮 TEXT DEMO 分布 (共 25490 条) ===
steps=3    →   1158 样本 ( 4.54%)
steps=4    →   4340 样本 (17.03%)
steps=5    →   5555 样本 (21.79%)
steps=6    →   3516 样本 (13.79%)
steps=7    →   3451 样本 (13.54%)
steps=8    →   2848 样本 (11.17%)
steps=9    →   2394 样本 ( 9.39%)
steps=10   →   1100 样本 ( 4.32%)
steps=11   →    363 样本 ( 1.42%)
steps=1

In [18]:
import json
from collections import defaultdict

file_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl"

# 初始化统计
visual_total = 0
text_total = 0
both_total = 0

visual_by_source = defaultdict(int)
text_by_source = defaultdict(int)
both_by_source = defaultdict(int)

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        ds = item.get("data_source", "unknown")
        has_visual = "visual_demo" in item
        has_text = "text_demo" in item

        if has_visual:
            visual_total += 1
            visual_by_source[ds] += 1
        if has_text:
            text_total += 1
            text_by_source[ds] += 1
        if has_visual and has_text:
            both_total += 1
            both_by_source[ds] += 1

# ===== 输出结果 =====
print("📊 === 全局统计 ===")
print(f"  有 visual_demo 的样本: {visual_total}")
print(f"  有 text_demo 的样本:   {text_total}")
print(f"  同时有 visual 和 text:  {both_total}")

print("\n📁 === 按 data_source 分类统计 ===")
sources = sorted(set(list(visual_by_source.keys()) + list(text_by_source.keys())))
for ds in sources:
    v = visual_by_source.get(ds, 0)
    t = text_by_source.get(ds, 0)
    b = both_by_source.get(ds, 0)
    print(f"  {ds:<25} → visual={v:<6} text={t:<6} both={b:<6}")


📊 === 全局统计 ===
  有 visual_demo 的样本: 98733
  有 text_demo 的样本:   25490
  同时有 visual 和 text:  0

📁 === 按 data_source 分类统计 ===
  h5_agilex_3rgb            → visual=41310  text=12064  both=0     
  h5_franka_3rgb            → visual=30168  text=9990   both=0     
  h5_tienkung_xsens_1rgb    → visual=10359  text=3436   both=0     
  h5_ur_1rgb                → visual=16896  text=0      both=0     


In [25]:
import json
import random
from collections import defaultdict, Counter

input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl"
output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_sampled_4to8.jsonl"

# 读取数据并整理为 {source -> {action -> [samples]}}
sources = defaultdict(lambda: defaultdict(list))

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        ds = item.get("data_source", "unknown")
        _id = item.get("id", "")
        parts = _id.split("/")
        if len(parts) < 2:
            continue
        action = parts[1]

        # total_steps 转换为整数
        steps = item.get("total_steps")
        if isinstance(steps, str):
            try:
                steps = int(steps)
            except ValueError:
                continue
        item["total_steps"] = steps

        # 仅保留 total_steps 在 [4,8] 的样本
        if not (4 <= steps <= 8):
            continue

        sources[ds][action].append(item)

# 计算每个 source 的 visual/text 比例
source_stats = {}
for ds, actions in sources.items():
    visual_count = 0
    text_count = 0
    for samples in actions.values():
        for s in samples:
            if "visual_demo" in s:
                visual_count += 1
            if "text_demo" in s:
                text_count += 1
    total = visual_count + text_count
    if total == 0:
        continue
    source_stats[ds] = {
        "visual_ratio": visual_count / total,
        "text_ratio": text_count / total,
        "total": total
    }

# 开始采样
target_total = 40000
total_source = len(source_stats)
avg_per_source = target_total // total_source

selected = []

for ds, actions in sources.items():
    # 当前 source 的目标比例
    ratios = source_stats.get(ds, {"visual_ratio": 1.0, "text_ratio": 0.0})
    n_vis = int(avg_per_source * ratios["visual_ratio"])
    n_txt = int(avg_per_source * ratios["text_ratio"])

    # 动作分类
    visual_actions = []
    text_actions = []

    for action, samples in actions.items():
        has_visual = any("visual_demo" in s for s in samples)
        has_text = any("text_demo" in s for s in samples)
        if has_visual:
            visual_actions.append((action, samples))
        if has_text:
            text_actions.append((action, samples))

    # 随机打乱动作
    random.shuffle(visual_actions)
    random.shuffle(text_actions)

    def pick_actions(actions_list, target_n):
        chosen = []
        for act, samples in actions_list:
            if len(chosen) >= target_n:
                break
            chosen.extend(samples)
        return chosen

    selected.extend(pick_actions(visual_actions, n_vis))
    selected.extend(pick_actions(text_actions, n_txt))

# 打乱结果
random.shuffle(selected)

# 写出结果
with open(output_path, "w", encoding="utf-8") as out:
    for item in selected:
        out.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ 严格筛选完成，最终采样 {len(selected)} 条样本（4 ≤ steps ≤ 8）")
print(f"📁 输出文件: {output_path}")


✅ 严格筛选完成，最终采样 35702 条样本（4 ≤ steps ≤ 8）
📁 输出文件: /projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_sampled_4to8.jsonl


In [26]:
# import json
# import random
# from collections import defaultdict

# input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl"
# output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all_visual_tiny.jsonl"

# # 读取数据
# data_by_source = defaultdict(list)

# with open(input_path, "r", encoding="utf-8") as f:
#     for line in f:
#         try:
#             item = json.loads(line)
#         except json.JSONDecodeError:
#             continue

#         # 只保留有 visual_demo 的样本
#         if "visual_demo" not in item:
#             continue

#         ds = item.get("data_source", "unknown")
#         steps = item.get("total_steps")
#         if isinstance(steps, str):
#             try:
#                 steps = int(steps)
#             except ValueError:
#                 continue
#         item["total_steps"] = steps
#         data_by_source[ds].append(item)

# # 按 data_source 和 step 分布取样
# selected = []
# for ds, items in data_by_source.items():
#     # 按 step 分组
#     step_groups = defaultdict(list)
#     for it in items:
#         step_groups[it["total_steps"]].append(it)

#     # 每种 step 数最多取 5 个
#     subset = []
#     for step_val, group in step_groups.items():
#         random.shuffle(group)
#         subset.extend(group[:5])

#     # 限制每个 source 最大取 50 条
#     random.shuffle(subset)
#     subset = subset[:50]
#     selected.extend(subset)

# # 如果超过200条，随机截断
# random.shuffle(selected)
# selected = selected[:200]

# # 写入输出文件
# with open(output_path, "w", encoding="utf-8") as out:
#     for item in selected:
#         out.write(json.dumps(item, ensure_ascii=False) + "\n")

# print(f"✅ 已生成 tiny visual demo 文件，共 {len(selected)} 条样本")
# print(f"📁 输出路径: {output_path}")


In [27]:
# import json
# import os

# input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all1.jsonl"
# output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/train/rl/rl_all.jsonl"

# count = 0

# with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
#     for line in infile:
#         line = line.strip()
#         if not line:
#             continue
#         try:
#             data = json.loads(line)
#         except json.JSONDecodeError:
#             continue

#         ds = data.get("data_source", "")
#         if ds.startswith("robomind_"):
#             data["data_source"] = ds.replace("robomind_", "", 1)
#             count += 1

#         outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

# print(f"✅ 修改完成！共更新 {count} 条记录。")
# print(f"输出文件: {output_path}")
